<a href="https://colab.research.google.com/github/lorenzosquadrani/convicity/blob/main/try_convicity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Libraries

In [1]:
!git clone https://github.com/lorenzosquadrani/convicity.git
!git clone https://github.com/lorenzosquadrani/plasticity.git
!mv /content/plasticity/plasticity/__version__.py.in /content/plasticity/plasticity/__version__.py

Cloning into 'convicity'...
remote: Enumerating objects: 63, done.
remote: Counting objects: 100% (63/63), done.
remote: Compressing objects: 100% (48/48), done.
remote: Total 63 (delta 20), reused 48 (delta 13), pack-reused 0
Unpacking objects: 100% (63/63), done.
Cloning into 'plasticity'...
remote: Enumerating objects: 1430, done.
remote: Counting objects: 100% (573/573), done.
remote: Compressing objects: 100% (369/369), done.
remote: Total 1430 (delta 366), reused 376 (delta 202), pack-reused 857
Receiving objects: 100% (1430/1430), 59.47 MiB | 29.52 MiB/s, done.
Resolving deltas: 100% (911/911), done.


In [2]:
cd plasticity/

/content/plasticity


In [10]:
from convicity.cBCM import cBCM

from plasticity.model.optimizer import SGD
from plasticity.model.weights import Normal
from plasticity.utils import view_weights

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchvision import datasets, transforms

import numpy as np
import matplotlib.pyplot as plt


#Load the Dataset

In [4]:
#load the data

trainset = datasets.MNIST('/content/convicity/', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ]))


testset  =  datasets.MNIST('/content/convicity/', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ]))

train_loader = torch.utils.data.DataLoader(trainset, batch_size=1000, shuffle=True)
test_loader = torch.utils.data.DataLoader(testset, batch_size=1000, shuffle=True)

# Model

In [5]:
class LittleNet(nn.Module):
    def __init__(self):
        super(LittleNet, self).__init__()
        self.flatten = nn.Flatten()
        self.conv_relu_stack = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=8, kernel_size=5, stride=1, padding=0)
            nn.ReLU()
        )
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(24*24, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        out = self.conv_relu_stack(x)
        out = self.flatten(x)
        out = self.linear_relu_stack(x)
        return out

SyntaxError: ignored

#Training

## Training convolutional layer using cBCM

In [6]:
from sklearn.datasets import fetch_openml

    #Download the MNIST dataset
print("Downloading the dataset...")
X, y = fetch_openml(name='mnist_784', version=1, data_id=None, return_X_y=True)

    # normalize the sample into [0, 1]
X *= 1. / 255

In [6]:
X = np.array(trainset.data)

model = cBCM(	n_filters = 8, kernel_size = 5,
				num_epochs= 10, batch_size = 1000, activation = 'relu',
				optimizer = SGD(lr=4e-2), weights_init = Normal(), interaction_strength = 0.,
				random_state = 42, verbose = True)

model.fit(X)
view_weights(model.weights, (5,5))

(60000, 24, 24, 5, 5)


  0%|          | 86/34560 [00:00<00:40, 857.32it/s]

Epoch 1/10


  0%|          | 103/34560 [00:00<00:33, 1027.92it/s]

Epoch 2/10


...

Epoch 4/10


  0%|          | 93/34560 [00:00<00:37, 927.39it/s]

Epoch 5/10


 11%|█▏        | 3938/34560 [00:03<00:31, 986.18it/s] 


KeyboardInterrupt: ignored

##Training the rest of the network

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))